# NequIP Tutorial 
This is an implementation of an NequIP energy and force prediction model in python. 
The trained model is deployed and integrated into LAMMPS MD engine to run an accelerated simulation on a single molecule.
Questions? How does the integration works? How to setup LAMMPS simulation? How to determine performance (accuracy) of the model?

In [56]:
# Import modules
import torch
torch.set_default_dtype(torch.float32)
import numpy as np
import logging
import pprint
from nequip.utils.config import Config
from nequip.train.trainer import Trainer
from nequip.data import AtomicDataDict
from nequip.data import AtomicData
from nequip.data import dataset_from_config
from ase.io import read, write

## Download and Visualize Molecule Dataset

In [57]:
# Remove existing sample folders and download molecule dataset
!rm -rf ./results/
!rm -rf ./benchmark_data
!rm *.zip
!rm *.npz
!mkdir benchmark_data
!curl http://quantum-machine.org/gdml/data/npz/toluene_ccsd_t.zip -o outfile.zip
!unzip outfile.zip
!y | rm -rf __MACOSX outfile outfile.zip
!mv toluene* ./benchmark_data
!ls benchmark_data


rm: cannot remove '*.zip': No such file or directory
rm: cannot remove '*.npz': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1020k  100 1020k    0     0   482k      0  0:00:02  0:00:02 --:--:--  482k
Archive:  outfile.zip
  inflating: toluene_ccsd_t-train.npz  
   creating: __MACOSX/
  inflating: __MACOSX/._toluene_ccsd_t-train.npz  
  inflating: toluene_ccsd_t-test.npz  
  inflating: __MACOSX/._toluene_ccsd_t-test.npz  
/usr/bin/sh: 1: y: not found
toluene_ccsd_t-test.npz  toluene_ccsd_t-train.npz


In [4]:
# Visualize
from ase.io import read
atoms = read('toluene.xyz', index=0)

from ase.visualize import view
view(atoms, viewer='x3d')

FileNotFoundError: [Errno 2] No such file or directory: 'toluene.xyz'

## Set torch geometric training dataset

In [58]:
# from nequip.data.dataset import NpzDataset
# dataset = NpzDataset('./tutorial_results/')
# # logging.info(f"Successfully loaded the data set of type {dataset}...")
npz_files = np.load('benchmark_data/toluene_ccsd_t-train.npz')
npz_files['E'].shape
npz_files['z']
npz_files['name']
print(npz_files.files)
print(npz_files['R'].shape)

['E', 'name', 'F', 'theory', 'R', 'z', 'type', 'md5']
(1000, 15, 3)


In [59]:
# Load config file
config = Config.from_file('./example.yaml')
pprint.pprint(config.as_dict())

{'BesselBasis_trainable': True,
 'PolynomialCutoff_p': 6,
 'append': True,
 'avg_num_neighbors': 'auto',
 'batch_size': 5,
 'chemical_symbols': ['H', 'C'],
 'conv_to_output_hidden_irreps_out': '16x0e',
 'dataset': 'npz',
 'dataset_file_name': './benchmark_data/toluene_ccsd_t-train.npz',
 'dataset_seed': 456,
 'dataset_url': 'http://quantum-machine.org/gdml/data/npz/toluene_ccsd_t.zip',
 'default_dtype': 'float32',
 'early_stopping_lower_bounds': {'LR': 1e-05},
 'early_stopping_patiences': {'validation_loss': 50},
 'ema_decay': 0.99,
 'ema_use_num_updates': True,
 'feature_irreps_hidden': '16x0o + 16x0e + 16x1o + 16x1e',
 'invariant_layers': 2,
 'invariant_neurons': 64,
 'irreps_edge_sh': '0e + 1o',
 'key_mapping': {'E': 'total_energy',
                 'F': 'forces',
                 'R': 'pos',
                 'z': 'atomic_numbers'},
 'l_max': 2,
 'learning_rate': 0.005,
 'log_batch_freq': 10,
 'log_epoch_freq': 1,
 'loss_coeffs': {'forces': 1, 'total_energy': [1, 'PerAtomMSELoss']},

In [60]:
# Instantiate Trainer object trainer with a config file. The trainer handles training as well as call back functions for logging, model saving, and early stopping.
trainer = Trainer(model=None, **dict(config))
trainer.kwargs

Torch device: cpu


{'root': 'results/toluene',
 'run_name': 'example-run-toluene',
 'append': True,
 'default_dtype': 'float32',
 'r_max': 4.0,
 'num_layers': 4,
 'l_max': 2,
 'parity': True,
 'num_features': 32,
 'nonlinearity_type': 'gate',
 'nonlinearity_scalars': {'e': 'silu', 'o': 'tanh'},
 'nonlinearity_gates': {'e': 'silu', 'o': 'tanh'},
 'num_basis': 8,
 'BesselBasis_trainable': True,
 'PolynomialCutoff_p': 6,
 'invariant_layers': 2,
 'invariant_neurons': 64,
 'avg_num_neighbors': 'auto',
 'use_sc': True,
 'dataset': 'npz',
 'dataset_url': 'http://quantum-machine.org/gdml/data/npz/toluene_ccsd_t.zip',
 'dataset_file_name': './benchmark_data/toluene_ccsd_t-train.npz',
 'key_mapping': {'z': 'atomic_numbers',
  'E': 'total_energy',
  'F': 'forces',
  'R': 'pos'},
 'npz_fixed_field_keys': ['atomic_numbers'],
 'chemical_symbols': ['H', 'C'],
 'num_types': 2,
 'feature_irreps_hidden': '16x0o + 16x0e + 16x1o + 16x1e',
 'irreps_edge_sh': '0e + 1o',
 'conv_to_output_hidden_irreps_out': '16x0e',
 'wandb': 

In [61]:
dataset = dataset_from_config(config, prefix='dataset')
logging.info(f"Successfully loaded the data set of type {dataset}...")
next(iter(dataset))

Processing dataset...
Loaded data: Batch(batch=[15000], cell=[1000, 3, 3], edge_cell_shift=[154352, 3], edge_index=[2, 154352], forces=[15000, 3], pbc=[1000, 3], pos=[15000, 3], ptr=[1001], total_energy=[1000, 1])
    processed data size: ~4.63 MB
Cached processed data to disk
Done!
Successfully loaded the data set of type NpzDataset(1000)...


AtomicData(atom_types=[15, 1], cell=[3, 3], edge_cell_shift=[152, 3], edge_index=[2, 152], forces=[15, 3], pbc=[3], pos=[15, 3], r_max=4.0, total_energy=[1])

In [62]:
trainer.set_dataset(dataset)
trainer.dataset_train.get_data()[0].keys()

dict_keys(['pos', 'total_energy', 'forces'])

In [63]:
# Normalize training dataset, compute statistics
(
    (forces_std,),
    (energies_mean, energies_std)
) = trainer.dataset_train.statistics(
    fields=[
        AtomicDataDict.FORCE_KEY,
        AtomicDataDict.TOTAL_ENERGY_KEY
    ],
    modes=["rms", "mean_std"],
)
print(f"Forces standard deviation: {forces_std.item()}")
print(f"Eneriges standard deiation: {energies_std.item()}")
print(f"Energies mean: {energies_mean.item()}")

Forces standard deviation: 30.621034622192383
Eneriges standard deiation: 6.097484111785889
Energies mean: -169793.3125


## Force and Energy Prediction

In [66]:
# Create model
from nequip.model._grads import ForceOutput
from typing import Optional
import logging
from e3nn import o3
from nequip.data import AtomicDataDict, AtomicDataset
from nequip.nn import (
    SequentialGraphNetwork,
    AtomwiseLinear,
    AtomwiseReduce,
    ConvNetLayer,
)
from nequip.nn.embedding import (
    OneHotAtomEncoding,
    RadialBasisEdgeEncoding,
    SphericalHarmonicEdgeAttrs,
)

from nequip.model import builder_utils

def EnergyModel(
    config, initialize: bool, dataset: Optional[AtomicDataset] = None
) -> SequentialGraphNetwork:
    """Base default energy model archetecture.

    For minimal and full configuration option listings, see ``minimal.yaml`` and ``example.yaml``.
    """
    logging.debug("Start building the network model")

    builder_utils.add_avg_num_neighbors(
        config=config, initialize=initialize, dataset=dataset
    )

    num_layers = config.get("num_layers", 3)

    layers = {
        # -- Encode --
        "one_hot": OneHotAtomEncoding,
        "spharm_edges": SphericalHarmonicEdgeAttrs,
        "radial_basis": RadialBasisEdgeEncoding,
        # -- Embed features --
        "chemical_embedding": AtomwiseLinear,
    }

    # add convnet layers
    # insertion preserves order
    for layer_i in range(num_layers):
        layers[f"layer{layer_i}_convnet"] = ConvNetLayer

    # .update also maintains insertion order
    layers.update(
        {
            # TODO: the next linear throws out all L > 0, don't create them in the last layer of convnet
            # -- output block --
            "conv_to_output_hidden": AtomwiseLinear,
            "output_hidden_to_scalar": (
                AtomwiseLinear,
                dict(irreps_out="1x0e", out_field=AtomicDataDict.PER_ATOM_ENERGY_KEY),
            ),
        }
    )

    layers["total_energy_sum"] = (
        AtomwiseReduce,
        dict(
            reduce="sum",
            field=AtomicDataDict.PER_ATOM_ENERGY_KEY,
            out_field=AtomicDataDict.TOTAL_ENERGY_KEY,
        ),
    )

    return SequentialGraphNetwork.from_parameters(
        shared_params=config,
        layers=layers,
    )

In [71]:
# Instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
energy_model = EnergyModel(config, initialize=True, dataset=dataset)
force_model = ForceOutput(energy_model)
print(force_model)

GradientOutput(
  (func): SequentialGraphNetwork(
    (one_hot): OneHotAtomEncoding()
    (spharm_edges): SphericalHarmonicEdgeAttrs(
      (sh): SphericalHarmonics()
    )
    (radial_basis): RadialBasisEdgeEncoding(
      (basis): BesselBasis()
      (cutoff): PolynomialCutoff()
    )
    (chemical_embedding): AtomwiseLinear(
      (linear): Linear(2x0e -> 2x0e | 4 weights)
    )
    (layer0_convnet): ConvNetLayer(
      (equivariant_nonlin): Gate (32x0e+16x1o -> 16x0e+16x1o)
      (conv): InteractionBlock(
        (linear_1): Linear(2x0e -> 2x0e | 4 weights)
        (fc): FullyConnectedNet[8, 64, 64, 4]
        (tp): TensorProduct(2x0e x 1x0e+1x1o -> 2x0e+2x1o | 4 paths | 4 weights)
        (linear_2): Linear(2x0e+2x1o -> 32x0e+16x1o | 96 weights)
        (sc): FullyConnectedTensorProduct(2x0e x 2x0e -> 32x0e+16x1o | 128 paths | 128 weights)
      )
    )
    (layer1_convnet): ConvNetLayer(
      (equivariant_nonlin): Gate (48x0e+16x1o+16x1e -> 16x0e+16x1o+16x1e)
      (conv): Inter

In [72]:
from nequip.nn import RescaleOutput
final_model = RescaleOutput(
    model=force_model,
    scale_keys=[AtomicDataDict.TOTAL_ENERGY_KEY, AtomicDataDict.FORCE_KEY],
    scale_by=forces_std,
    shift_keys=AtomicDataDict.TOTAL_ENERGY_KEY,
    shift_by=energies_mean,
).to(device)

In [ ]:
# Training
trainer.model = final_model
trainer.train()

! Restarting training ...

training
# Epoch batch         loss       loss_f       loss_e        f_mae       f_rmse      H_f_mae      C_f_mae  psavg_f_mae     H_f_rmse     C_f_rmse psavg_f_rmse        e_mae      e/N_mae
     28    10            1        0.992      0.00891         22.5         30.5         15.2         30.7           23         20.7         38.8         29.7           43         2.87
     28    20        0.979         0.97      0.00927         22.6         30.2         16.8         29.2           23         22.6           37         29.8         43.4         2.89

validation
# Epoch batch         loss       loss_f       loss_e        f_mae       f_rmse      H_f_mae      C_f_mae  psavg_f_mae     H_f_rmse     C_f_rmse psavg_f_rmse        e_mae      e/N_mae
     28     5        0.953        0.943       0.0098         22.1         29.7         15.4         29.8         22.6         20.5         37.6         29.1         45.2         3.01


  Train      #    Epoch      wal   

## Inference

In [ ]:
# Inference on last frame in dataset
toluene_data = np.load(config.dataset_file_name)
r = toluene_data['R'][-1]
forces = toluene_data['F'][-1]
ATOMIC_NUMBERS_KEY = torch.Tensor(torch.from_numpy(toluene_data['z'].astype(np.float32))).to(torch.int64)

data = AtomicData.from_points(
    pos=r,
    r_max=config['r_max'], 
    **{AtomicDataDict.ATOMIC_NUMBERS_KEY: ATOMIC_NUMBERS_KEY,
    AtomicDataDict.ATOM_TYPE_KEY: torch.zeros_like(ATOMIC_NUMBERS_KEY)
    }
)

In [ ]:
force_model.eval(); 
pred = force_model(AtomicData.to_AtomicDataDict(data))['forces']

In [ ]:
# Plot original and predicted force vectors
from matplotlib import pyplot as plt
plt.figure(figsize=(12, 12))

plt.plot(
    r[:, 0],
    r[:, 1],
    '.k',
    markersize=10,
)

plt.quiver(
    r[:, 0],
    r[:, 1],
    pred[:, 0].detach().numpy(),
    pred[:, 1].detach().numpy(),
    norm=None
)

plt.quiver(
    r[:, 0], 
    r[:, 1], 
    forces[:, 0],
    forces[:, 1],
    color='red', 
    norm=None
)

plt.legend(['Positions', 'Predicted', 'True'], prop={'size': 20})
plt.show()

## MD Simulation with LAMMPS

In [ ]:
# # Compile lammps
# !wget "https://github.com/lammps/lammps/archive/stable.zip"
# !unzip -q stable.zip
# !rm stable.zip
# !mv lammps-stable lammps
# !wget "https://github.com/mir-group/pair_nequip/archive/main.zip"
# !unzip -q main.zip
# !rm main.zip
# !mv pair_nequip-main pair_nequip
# !cd pair_nequip && ./patch_lammps.sh ../lammps


In [1]:
# compile lammps
!git clone -b stable_29Sep2021_update2 --depth 1 https://github.com/lammps/lammps.git
!wget "https://github.com/mir-group/pair_nequip/archive/main.zip"
!unzip -q main.zip
!rm main.zip
!mv pair_nequip-main pair_nequip
!cd pair_nequip && ./patch_lammps.sh ../lammps
# !pip install mkl mkl-include
!cd lammps && mkdir -p build && cd build && cmake ../cmake -D CMAKE_PREFIX_PATH=`python -c 'import torch;print(torch.utils.cmake_prefix_path)'` && make -j4

Cloning into 'lammps'...
remote: Enumerating objects: 11732, done.
remote: Counting objects: 100% (11732/11732), done.
remote: Compressing objects: 100% (8603/8603), done.
remote: Total 11732 (delta 3943), reused 6318 (delta 2930), pack-reused 0
Receiving objects: 100% (11732/11732), 110.00 MiB | 12.46 MiB/s, done.
Resolving deltas: 100% (3943/3943), done.
Note: switching to '7586adbb6a61254125992709ef2fda9134cfca6c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

Updating files: 100% (11058/11058), done.
--2023-03-22 10:50:

In [ ]:
%pip install mkl mkl-include

In [ ]:
import torch
print(torch.utils.cmake_prefix_path)

In [ ]:
# mkl install not working with pip
%pip install mkl mkl-include
!cd lammps && mkdir -p build && cd build && cmake ../cmake -DCMAKE_PREFIX_PATH=`python -c 'import torch;print(torch.utils.cmake_prefix_path)'` && make -j4


In [ ]:
# how to setup lammps config file?
lammps_input_minimize = """
units	real
atom_style atomic
newton off
thermo 1
read_data structure.data

pair_style	nequip
pair_coeff	* * ../toluene-deployed.pth C H 
mass            1 15.9994
mass            2 1.00794

neighbor 1.0 bin
neigh_modify delay 5 every 1

minimize 0.0 1.0e-8 10000 1000000
write_dump all custom output.dump id type x y z fx fy fz
"""
!mkdir lammps_run
with open("lammps_run/toluene_minimize.in", "w") as f:
    f.write(lammps_input_minimize)

In [ ]:
toluene_example = """15
 Lattice="100.0 0.0 0.0 0.0 100.0 0.0 0.0 0.0 100.0" Properties=species:S:1:pos:R:3 -169777.5840406276=T pbc="F F F"
 C       52.48936904      49.86911725      50.09520748
 C       51.01088202      49.89609925      50.17978049
 C       50.36647401      50.04650925      48.96054247
 C       48.95673398      50.29576626      48.71580846
 C       48.04533296      50.26023426      49.82589448
 C       48.70932398      49.85770925      51.01923950
 C       50.06326400      49.77782925      51.25691751
 H       52.94467905      50.48672926      50.86545150
 H       52.89060405      48.87175023      50.14480949
 H       53.02173405      50.05890725      49.03968247
 H       51.01439802      50.38234726      48.05314045
 H       48.80598498      50.64314926      47.68195744
 H       46.96754695      50.20586626      49.53998848
 H       48.16716997      49.75850325      51.88622952
 H       50.45791001      49.55387424      52.15303052
 """

with open('toluene.xyz', 'w') as f: 
    f.write(toluene_example)

In [ ]:
# Read as ASE objects
atoms = read('toluene.xyz', format='extxyz')

# Perturb positions
p = atoms.get_positions()
p += np.random.rand(15, 3) * 0.5
atoms.set_positions(p)
atoms.set_pbc(False)

# Write to a LAMMPS file
write("lammps_run/structure.data", atoms, format="lammps-data")

In [ ]:
# from ase.io import read
# atoms = read('toluene.xyz', index=0)
from ase.visualize import view
view(atoms, viewer='x3d')

In [ ]:
!cd lammps_run/ && ../lammps/build/lmp -in toluene_minimize.in